In [3]:
from hashlib import sha256
import base58

# Bitcoin address manipulation  - 
## Why you should check every character in an address when signing a transaction.

Generating a valid bitcoin address that shares most of the characters with another address is not difficult if you don't care about having the private key. This allows an attacker to cause you to lose funds with a relatively trivial amount of work.

At it's most difficult, you can generate a different valid address by changing any 6 non-sequential characters in 10's of hours using this notebook and a standard laptop. If you fix the last 4 characters and up to the first 25 charaters, it it significantly easier (~2-3min to generate a valid malicious address). Fixing the final 6 characters is equivalent to the most difficult case.

The goal of this post is to demonstrate why it’s important to verify every character in an address, not just the first and last few values, by showing how easy it is for an adversary to generate an address that looks similar to a valid address generated by your wallet. While such an attack couldn’t be used to steal funds, it would still result in lost bitcoin. 

I’ll also cover some of the most important mechanics behind bitcoin address serialization, including Base58 encoding and checksums.

All code for this post is written in python. You can run it yourself from the jupyter notebook, available on GitHub [here](https://github.com/destrys/address_manipulation).


**NOTE:** For this example, I will be using this address: `1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ`, it was randomly grabbed from a
block explorer, apologies if it is yours.

## Base58 Basics

Before I explain address serialization, I must first explain Base58, and give some examples of converting between bases.

When we talk about the “base” of a number, what we’re referring to is the formatting used to represent that data. This is also known as [Positional Notation](https://en.wikipedia.org/wiki/Positional_notation). Most recognize the term “binary”, which is how we can understand computers to interpret data. Binary is a base 2 representation, i.e. 1s and 0s (bi - two numbers). The decimal system is base 10 (deci - ten, i.e. 0-9) and [hexadecimal](https://en.wikipedia.org/wiki/Hexadecimal) is base 16 which uses the numbers 0-9 and letters A-F, generally used to present data in a more human readable format.

Base58 uses 58 characters: 
(`123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz`)

> In contrast to Base32 or Base64, the digits of the encoding do not line up well with byte boundaries of the original data. For this reason, the method is well-suited to encode large integers, but not designed to encode longer portions of binary data.

Because 58 is not a power of two, the characters in base58 don't correspond to a fixed number of bytes.
Decimal has the same problem, one byte has 256 values, but that can require 1, 2, or 3 decimal characters.

Because of this, when converting a number to base58 from any other standard base, it may be impossible to know what the least significant digits will be, as they are dependent on all the digits in the input. (Unfortunately I couldn’t find a reference for this, so if any readers can help verify it would be greatly appreciated.)

The most significant digits, however, are not dependendent on all the digits, only the most significant ones.
If you’re curious to learn more about what we’re talking about when we reference most and least significant digits, the best way is to learn about “Endianness” which you can see [here](https://en.wikipedia.org/wiki/Endianness).

An example might help explain this:

In [4]:
a_big_number = 1234567890

Then we’ll convert it into 4 different types: decimal (should be the same), hexadecimal, bytes (as a list of integers), and base58. 

In [6]:
print("Decimal: ", a_big_number, " Length: ", len(str(a_big_number)))
a_big_hex_number = str(hex(a_big_number))[2:]
print("Hexidecimal: ", a_big_hex_number, " Length: ", len(a_big_hex_number))
a_big_byte_number = a_big_number.to_bytes(4,byteorder='big')
print("Bytes:", list(a_big_byte_number), " Length: ", len(a_big_byte_number))
a_big_base58_number = base58.b58encode(a_big_number.to_bytes(4,byteorder='big'))
print("base58: ", a_big_base58_number.decode('ascii'), " Length: ", len(a_big_base58_number))
a_big_base58_number = base58.b58encode_int(a_big_number)
print("base58 from int: ", a_big_base58_number.decode('ascii'), " Length: ", len(a_big_base58_number))

Decimal:  1234567890  Length:  10
Hexidecimal:  499602d2  Length:  8
Bytes: [73, 150, 2, 210]  Length:  4
base58:  2t6V2H  Length:  6
base58 from int:  2t6V2H  Length:  6


Let’s change the least significant digit now, the one on the right side (the `0`), and see what happens.

In [7]:
a_big_number = 1234567891
print("Decimal: ", a_big_number, " Length: ", len(str(a_big_number)))
a_big_hex_number = str(hex(a_big_number))[2:]
print("Hexidecimal: ", a_big_hex_number, " Length: ", len(a_big_hex_number))
a_big_byte_number = a_big_number.to_bytes(4,byteorder='big')
print("Bytes:", list(a_big_byte_number), " Length: ", len(a_big_byte_number))
a_big_base58_number = base58.b58encode(a_big_number.to_bytes(4,byteorder='big'))
print("base58: ", a_big_base58_number.decode('ascii'), " Length: ", len(a_big_base58_number))

Decimal:  1234567891  Length:  10
Hexidecimal:  499602d3  Length:  8
Bytes: [73, 150, 2, 211]  Length:  4
base58:  2t6V2J  Length:  6


Let’s next change one of the most significant digits (on the left side) and see what the result is.

In [10]:
a_big_number = 1334567890
print("Decimal: ", a_big_number, " Length: ", len(str(a_big_number)))
a_big_hex_number = str(hex(a_big_number))[2:]
print("Hexidecimal: ", a_big_hex_number, " Length: ", len(a_big_hex_number))
a_big_byte_number = a_big_number.to_bytes(4,byteorder='big')
print("Bytes:", list(a_big_byte_number), " Length: ", len(a_big_byte_number))
a_big_base58_number = base58.b58encode(a_big_number.to_bytes(4,byteorder='big'))
print("base58: ", a_big_base58_number.decode('ascii'), " Length: ", len(a_big_base58_number))

Decimal:  1334567890  Length:  10
Hexidecimal:  4f8be3d2  Length:  8
Bytes: [79, 139, 227, 210]  Length:  4
base58:  32w1YD  Length:  6


Notice that when we change the least significant digit, only the least significant digit changes for the other bases. But if we change a highly significant digit, all the other digits change in the other bases. This is because one decimal digit doesn't map cleanly to a fixed number of digits in any of the other bases. Since bytes and hex do map cleanly, you can change a highly significant digit and not affect the less significant digits in the other base, as you can see next:

In [12]:
print('hex', '     bytes', '              binary', '                         decimal')
hex_number = "4f8be3d2"
print(hex_number, list(bytes.fromhex(hex_number)), "{0:b}".format(int(hex_number,16)), int(hex_number,16))
hex_number_2 = "5f8be3d2"
print(hex_number_2, list(bytes.fromhex(hex_number_2)), "{0:b}".format(int(hex_number_2,16)), int(hex_number_2,16))

hex      bytes               binary                          decimal
4f8be3d2 [79, 139, 227, 210] 1001111100010111110001111010010 1334567890
5f8be3d2 [95, 139, 227, 210] 1011111100010111110001111010010 1603003346


## Bitcoin Addresses
Now let's talk about bitcoin addresses. A bitcoin address has three components: 
- The version byte
- The data
- The Checksum

### The Version Byte
The version byte gives us information about the context of the address.

> 0x00 for P2PKH addresses on the main Bitcoin network (mainnet)  
> 0x6f for P2PKH addresses on the Bitcoin testing network (testnet)  
> 0x05 for P2SH addresses on mainnet  
> 0xc4 for P2SH addresses on testnet  

This gives wallets and other related software the ability to check that the address is being used where it’s expected. For example, if you paste in an address with the version byte `0x6f` into a mainnet wallet, the wallet will be able to show an error, letting you know that you probably don’t want to send funds to an address meant for testnet. 


### The Data
This will depend on the type of address you’re generating, but for a P2PKH address, the data portion of an address is the hash (actually hashed twice: RIPEMD(SHA256(pubkey))) of a public key.

### The Checksum
Bitcoin Addresses are actually Base58Check encoded, **not** simply Base58. 

A checksum is used in computing for data validation. For an address, this can detect typos by comparing the data to the checksum and making sure we get what we expected.

In Base58Check, you first double-hash the version byte and data with sha256. You take the first 4 bytes of the resulting hash and append it to the data, and then you convert the full (version byte + data + checksum) byte array into Base58.

### A Note on Checksum collisions

By only taking the first 4 bytes of the hash, the checksum field is far from immune to collisions. SHA256 properly generates any even distribution of outputs. 4 bytes can only encode $256^4$ values (4,294,967,296), so if you want to find a hash collision, you only need to try an expected 4.3 Billion values per collision. This is important for this discussion, but it isn't the end of the story.

You might think at this point, "Cool, so if I want to create a malicious bitcoin address, I'll just decode it to bytes, and start changing bytes until I find a checksum collision, and I'm set!" But you'd be wrong. If you skimmed the Base58 discussion above, maybe take a closer read. **If you change some bytes in the middle of an address, when you go to encode it in base58, all of the lower significant digits are modified in base58.**

So now you might think: "Shit, so I have to modify the data, find a checksum collision, and then hope it encodes to the same last 4 digits? I'm screwed, bitcoin addresses are hard."

It's true, bitcoin addresses are confusing, but no, it's not that difficult to create a malicious address, and that's because while the checksum has to be *valid*, it doesn't have to be the *same* checksum of the original address.

### Generating a Malicious address.

Let's start simple. Let's say you want a valid bitcoin address with the same $A$ number of characters at the beginning, and the last digit to be the same. Let's assume that the resulting addresses will have an even distibution of final characters, since we're going to be changing a bunch of digits (some in the data, and all of the checksum).

If it's an even distribution, for each value we try, we have a 1-in-58 probability that it will be the correct value.
Let's give it a try!

We'll try every value in one byte, that's 256 values, but the last character can be only one of the 58 characters in Bse58, so we should expect to see 256/58 = 4.4 matches

In [15]:
# Note for the nerds: I'm not optimizing the code, instead I'm favoring readability.
def cycle_through_one_byte(ver, data, digit, match):
    dataarray = bytearray(data)
    for i in range(256):
        # Change a byte
        dataarray[digit] = i
        # Compute the double sha256 hash and take the first 4 bytes
        check1 = sha256(sha256(ver+dataarray).digest()).digest()[:4]
        # convert to base58
        addr = base58.b58encode(ver+dataarray+check1)
        # Check the last digit
        if (addr[-1:] == match):
            print('found: ', addr.decode('ascii'))

In [16]:
original_address ='1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ'
k = base58.b58decode(original_address)
verbyte, data, check0 = k[0:1], k[1:-4], k[-4:]

In [17]:
cycle_through_one_byte(verbyte, data, 19, b'Z')

found:  1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFxYGxT7Z
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFyeGfzQZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cG438ERVZ


In [18]:
cycle_through_one_byte(verbyte, data, 19,  b'1')

found:  1HU1LDBXUg73f2ro2e2dB3XY8cFkFSmCQ1
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFwPyAZ91
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFysN4xF1
found:  1HU1LDBXUg73f2ro2e2dB3XY8cG4pxmyY1
found:  1HU1LDBXUg73f2ro2e2dB3XY8cGB5Rhk51


In [19]:
%time cycle_through_one_byte(verbyte, data, 19, b'2')

found:  1HU1LDBXUg73f2ro2e2dB3XY8cFnB2tYx2
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFqTayv52
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFzcYYU22
found:  1HU1LDBXUg73f2ro2e2dB3XY8cG2q3M6u2
found:  1HU1LDBXUg73f2ro2e2dB3XY8cG4RpLuB2
CPU times: user 4.57 ms, sys: 680 µs, total: 5.25 ms
Wall time: 4.68 ms


Looks pretty solid. I think this is dependent on which digit you're changing, though I don't know how to generalize at this time. For example, if you change the 10th byte instead of the 19th:

In [20]:
cycle_through_one_byte(verbyte, data, 10, b'Z')

found:  1HU1LDBXUg73f2rS5EwnM2GQLJizdsKtzZ
found:  1HU1LDBXUg73f2rZo25fuygJ3nYycfXiyZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ
found:  1HU1LDBXUg73f2rqEaMT3tW5UkCwaKM2oZ
found:  1HU1LDBXUg73f2rw7jtKPUSBjn5Jr7ykzZ
found:  1HU1LDBXUg73f2rzReNZCjtzkz11r3GvJZ
found:  1HU1LDBXUg73f2rzny6MrDPkefzYDh1k1Z
found:  1HU1LDBXUg73f2s73TM2qGpcoPrRqdNtSZ
found:  1HU1LDBXUg73f2sKYRrNoPgM6qaDAqb3MZ
found:  1HU1LDBXUg73f2sf2Y413WxTLP8vqTGk6Z


Notice how the addresses look significantly different from each other when not changing the least significant byte.

But if we want the last digit to be a `1` there’s only one match.

In [22]:
cycle_through_one_byte(verbyte, data, 10, b'1')

found:  1HU1LDBXUg73f2sgs9f2GtSEnq6YQ4sgH1


Though maybe that's just statistics being fun.

Let's move on to 2 digit matches. With 2 digits we have $58^2$ so 3,364 options, But if we cycle through all values in 2 bytes, that's $256^2$ or 65,536. So we should expect to see, 19.5 matches

In [27]:
def cycle_through_two_bytes(ver, data, match):
    total_matches = 0
    dataarray = bytearray(data)
    for i in range(256):
        # Change a byte
        dataarray[19] = i
        for j in range(256):
            dataarray[18] = j
            # Compute the double sha256 hash and take the first 4 bytes
            check1 = sha256(sha256(ver+dataarray).digest()).digest()[:4]
            # convert to base58
            addr = base58.b58encode(ver+dataarray+check1)
            # CHeck the last two digits
            if (addr[-2:] == match):
                total_matches+=1
                print('found: ', addr.decode('ascii'))
    print("Total Matches:", total_matches)

In [28]:
cycle_through_two_bytes(verbyte, data, b'ZZ')

found:  1HU1LDBXUg73f2ro2e2dB3XY8bhL6FNdZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8b5VnZNvZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cY9otddZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cQhSD81ZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8aq5LtioZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8ay3kBjzZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8an97xSxZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cXKEfctZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bd2iMNCZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8b99mNuQZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bt1pxGTZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8c7zDqXXZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cBVPbUQZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8b1nWCCgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cSTME4jZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bp7Nc5dZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8c549A7UZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bxasQpoZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bRE69KdZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bVDxYCzZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8agwJoHTZZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8aZWPiFsZZ
found:  1HU

In [29]:
cycle_through_two_bytes(verbyte, data, b'aZ')

found:  1HU1LDBXUg73f2ro2e2dB3XY8ae7QdPKaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bdPxVDTaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bJzuAiZaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8b99L4dDaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bb3swQDaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bm3A3i8aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bSekXz2aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8cAVywr3aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bLif2joaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bAmk3KXaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8beCoj1JaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8aZy1QL2aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bwAkqDYaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bmiDSWGaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8c1CFZ9waZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bhH2ykPaZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8ayy1yF9aZ
found:  1HU1LDBXUg73f2ro2e2dB3XY8bPtonGxaZ
Total Matches: 18


In [30]:
%time cycle_through_two_bytes(verbyte, data, b'11')

found:  1HU1LDBXUg73f2ro2e2dB3XY8cQfuTxL11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bvp6pgm11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bkN6vHT11
found:  1HU1LDBXUg73f2ro2e2dB3XY8az3nv1o11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bMVM9AF11
found:  1HU1LDBXUg73f2ro2e2dB3XY8c1rC2wg11
found:  1HU1LDBXUg73f2ro2e2dB3XY8btWHoTC11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cWrnG9d11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bR8RfEb11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bEgqkAu11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cht15Xm11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cGVVvF811
found:  1HU1LDBXUg73f2ro2e2dB3XY8agtFyKQ11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bjfNS2s11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bPkcyca11
found:  1HU1LDBXUg73f2ro2e2dB3XY8bGniJ6M11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cM4y1rG11
found:  1HU1LDBXUg73f2ro2e2dB3XY8biGTxzX11
found:  1HU1LDBXUg73f2ro2e2dB3XY8akWcvRn11
found:  1HU1LDBXUg73f2ro2e2dB3XY8cXd3wQH11
Total Matches: 20
CPU times: user 968 ms, sys: 12.6 ms, total: 981 ms
Wall time: 977 ms


Pretty solid so far. Notice that I'm changing only the least significant byte in the data section. This means the whole first section of address is unchanged.

Let's move on to 3 digits.

With 3 digits we have $58^3$ so 195,112 options, But if we cycle through all values in 3 bytes, that's $256^3$ or 16,777,216. So we should expect to see, 86 matches

In [31]:
def cycle_through_three_bytes(ver, data, match):
    total_matches = 0
    dataarray = bytearray(data)
    for i in range(256):
        # Change a byte
        dataarray[19] = i
        for j in range(256):
            dataarray[18] = j
            for k in range(256):
                dataarray[17] = k
                # Compute the double sha256 hash and take the first 4 bytes
                check1 = sha256(sha256(ver+dataarray).digest()).digest()[:4]
                # convert to base58
                addr = base58.b58encode(ver+dataarray+check1)
                # CHeck the last digit
                if (addr[-3:] == match):
                    total_matches+=1
                    #print('found: ', addr)
    print("Total Matches:", total_matches)

In [32]:
print(original_address)

1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ


In [33]:
%time cycle_through_three_bytes(verbyte, data, b'gZZ')

Total Matches: 79
CPU times: user 4min 45s, sys: 422 ms, total: 4min 46s
Wall time: 4min 46s


Along with agreeing with our statistics so far, notice that the execution time has grown linearly with number of operations. 8ms for one byte, 1s for two bytes (256x the ops of one byte), 305s for three bytes (256x the ops of two bytes)

Let's move on to 4 digits. Which for some reason is a standard, "check the first and last 4". This is probably due to misunderstanding the checksum field. 4 base58 digits can only encode 3 bytes worth of data $(58^4 < 256^3)$, and as mentioned earlier, you can't decode the least significant digits only, you need the full value to decode.

With 4 digits we have $58^4$ so 11,316,496 options, But we could cycle through all values in only 3 bytes because that's $256^3$ or 16,777,216. So we should expect to see, 1.5 matches.

We can modifed a fourth byte, but instead of using all 256 available values, just use 4 values. This will boost the expect matches to 6 (scanning over $256^3 * 4$ or 67,108,864 values) while expecting ~20min execution time.

In [21]:
def cycle_through_four_bytes(ver, data, match):
    total_matches = 0
    dataarray = bytearray(data)
    for i in range(4):
        # Change a byte
        dataarray[19] = i
        for j in range(256):
            dataarray[18] = j
            for k in range(256):
                dataarray[17] = k
                for l in range(256):
                    dataarray[16] = l
                    # Compute the double sha256 hash and take the first 4 bytes
                    check1 = sha256(sha256(ver+dataarray).digest()).digest()[:4]
                    # convert to base58
                    addr = base58.b58encode(ver+dataarray+check1)
                    # CHeck the last digit
                    if (addr[-4:] == match):
                        total_matches+=1
                        print('found: ', addr.decode('ascii'))
    print("Total Matches:", total_matches)

In [22]:
print(original_address)

1HU1LDBXUg73f2ro2e2dB3XY8cFoYLFgZZ


In [23]:
%time cycle_through_four_bytes(verbyte, data, b'FgZZ')

found:  1HU1LDBXUg73f2ro2e2dB3YAZkLzhmFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3YAsH6C6bFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XeYUj3vkFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3Xx2JDVAbFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3Y6ZfhPG1FgZZ
found:  1HU1LDBXUg73f2ro2e2dB3Y5QbkmDLFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3YEXcH33JFgZZ
found:  1HU1LDBXUg73f2ro2e2dB3XhYFrPERFgZZ
Total Matches: 8
CPU times: user 18min 12s, sys: 699 ms, total: 18min 12s
Wall time: 23min 49s


Great. So we can create duplicate valid first4...last4 bitcoin addresses at a rate of once every 3 minutes or so.

Now what about fixing the last 5 digits? 

Now we have $58^5$ = 656,356,768, which is 40x more than 3 bytes, but still well less than the full $256^4$ (4,294,967,296). That'll be roughly 3 hours of execution to generate one, which, while being more than feasible using off-the-shelf hardware, is more than I’m willing to put my computer through at the moment, so we'll leave that as an exercise for you to try at home if you’re interested.

And what about 6 or 7 digits?

6 digits gives us a 1 in 38,068,692,544 chance to find a valid address. That's interesting because now we're above $256^4$ which is the hash-collision scale. At this point, I think it will be faster to invert this process.

Instead of searching for matches to our base58 pattern, let's search through base58 address values to find valid addresses.
A given base58 string should have a 1-in-4,294,967,296 chance of being valid $(256^4)$.

Based on our earlier benchmarks, that should take roughly 20 hours of execution (4200M tries per collision / 3M tries per minute $\approx$ 20hrs) . To get to that number of options, we would need to be able to change 6 digits of the address, and I don't think they need to be contiguous.

I'll try this in the future and update.

## What Did We Learn

# Bech32

[BIP173](https://github.com/bitcoin/bips/blob/master/bip-0173.mediawiki) introduced a new addresses encoding for bitcoin addresses that use native segwit: bech32. If you guessed that bech32 is a base 32 encoding, you're right. If you noticed that 32 is a power of 2 ($2^5$) you may also guess that this would make modifying bech32 addresses easier, and you're right again!

In [35]:
import bech32
import itertools

In [2]:
from bech32 import CHARSET

In [3]:
len(CHARSET)

32

In [11]:
print(''.join(sorted(list(CHARSET))))

023456789acdefghjklmnpqrstuvwxyz


In [37]:
an_address = 'bc1pw508d6qejxtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k7grplx'

In [29]:
hrp, data = bech32.bech32_decode(an_address)
print(hrp)
print(data)

bc
[1, 14, 20, 15, 7, 13, 26, 0, 25, 18, 6, 11, 13, 8, 21, 4, 20, 3, 17, 2, 29, 3, 12, 29, 3, 4, 15, 24, 20, 6, 14, 30, 22, 14, 20, 15, 7, 13, 26, 0, 25, 18, 6, 11, 13, 8, 21, 4, 20, 3, 17, 2, 29, 3, 12, 29, 3, 4, 15, 24, 20, 6, 14, 30, 22]


In [30]:
data[4] = 0

In [31]:
bech32.bech32_encode(hrp, data)

'bc1pw50qd6qejxtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7knnepau'

In [32]:
bech32.bech32_create_checksum(hrp, data)

[19, 19, 25, 1, 29, 28]

In [34]:
''.join([CHARSET[d] for d in data])

'pw50qd6qejxtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k'

In [36]:
''.join([CHARSET[d] for d in bech32.bech32_create_checksum(hrp, data)])

'nnepau'

In [45]:
def cycle_through_one_char_and_match_one(address, digit):
    hrp, data = bech32.bech32_decode(an_address)
    checksum = bech32.bech32_create_checksum(hrp, data)
    for i in range(32):
        # Change a byte
        data[digit] = i
        # Compute the checksum
        check1 = bech32.bech32_create_checksum(hrp, data)
        # Check the last digit
        if (check1[-1:] == checksum[-1:]):
            print('found: ', bech32.bech32_encode(hrp, data))
                
def cycle_through_two_char_and_match_one(address, digits):
    hrp, data = bech32.bech32_decode(an_address)
    checksum = bech32.bech32_create_checksum(hrp, data)
    for values in itertools.product(range(32), repeat=2):
        # Change the bytes
        for indx, digit in enumerate(digits):
            data[digit] = values[indx]
        # Compute the checksum
        check1 = bech32.bech32_create_checksum(hrp, data)
        # Check the last digit
        if (check1[-1:] == checksum[-1:]):
            print('found: ', bech32.bech32_encode(hrp, data))

In [46]:
print(an_address)
%time cycle_through_one_char_and_match_one(an_address, 8)

bc1pw508d6qejxtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k7grplx
found:  bc1pw508d6qejxtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k7grplx
CPU times: user 6.44 ms, sys: 181 µs, total: 6.62 ms
Wall time: 6.51 ms


In [47]:
print(an_address)
%time cycle_through_two_char_and_match_one(an_address, [8, 10])

bc1pw508d6qejxtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k7grplx
found:  bc1pw508d6qqj8tdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kshclgx
found:  bc1pw508d6qpjjtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7ken6wqx
found:  bc1pw508d6qzjytdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kzlu5cx
found:  bc1pw508d6qrj3tdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7ktm79sx
found:  bc1pw508d6qyjptdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7ka8sfpx
found:  bc1pw508d6q9j5tdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k5rjcfx
found:  bc1pw508d6qxjztdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k005z3x
found:  bc1pw508d6q8jhtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kxtknex
found:  bc1pw508d6qgjttdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k27g66x
found:  bc1pw508d6qfj7tdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kr62tjx
found:  bc1pw508d6q2jgtdg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kckv32x
found:  bc1p

In [48]:
def cycle_through_three_char_and_match_two(address, digits):
    hrp, data = bech32.bech32_decode(an_address)
    checksum = bech32.bech32_create_checksum(hrp, data)
    for values in itertools.product(range(32), repeat=3):
        # Change the bytes
        for indx, digit in enumerate(digits):
            data[digit] = values[indx]
        # Compute the checksum
        check1 = bech32.bech32_create_checksum(hrp, data)
        # Check the last two digits
        if (check1[-2:] == checksum[-2:]):
            print('found: ', bech32.bech32_encode(hrp, data))

In [50]:
%time cycle_through_three_char_and_match_two(an_address, [8, 10, 12])

found:  bc1pw508d6qqj6t9g4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7krrerlx
found:  bc1pw508d6qpjgtfg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kyewqlx
found:  bc1pw508d6qzjhtag4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kd779lx
found:  bc1pw508d6qrj9t3g4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k2yfxlx
found:  bc1pw508d6qyjqtug4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7klsh0lx
found:  bc1pw508d6q9jjtsg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kc2qvlx
found:  bc1pw508d6qxjdtyg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k3dsflx
found:  bc1pw508d6q8jltgg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kkh82lx
found:  bc1pw508d6qgj8t7g4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kjv9mlx
found:  bc1pw508d6qfj4tjg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k4kjclx
found:  bc1pw508d6q2j2txg4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7ku3zalx
found:  bc1pw508d6qtjct2g4y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kmt47lx
foun

In [53]:
def cycle_through_four_char_and_match_three(address, digits):
    hrp, data = bech32.bech32_decode(an_address)
    checksum = bech32.bech32_create_checksum(hrp, data)
    for values in itertools.product(range(32), repeat=4):
        # Change the bytes
        for indx, digit in enumerate(digits):
            data[digit] = values[indx]    
        # Compute the checksum
        check1 = bech32.bech32_create_checksum(hrp, data)
        # Check the last digit
        if (check1[-3:] == checksum[-3:]):
            print('found: ', bech32.bech32_encode(hrp, data))

In [54]:
%time cycle_through_four_char_and_match_three(an_address, [8, 10, 12, 14])

found:  bc1pw508d6qqjftlgmy5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kuhwplx
found:  bc1pw508d6qpj4tygjy5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kln3plx
found:  bc1pw508d6qzjctqgfy5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k6leplx
found:  bc1pw508d6qrjytmgqy5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kemxplx
found:  bc1pw508d6qyjztggky5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7ks8fplx
found:  bc1pw508d6q9j7tngly5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7knrkplx
found:  bc1pw508d6qxjnthgyy5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kk07plx
found:  bc1pw508d6q8j0tvgdy5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k4tpplx
found:  bc1pw508d6qgjltcgpy5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7ky7qplx
found:  bc1pw508d6qfjrtrggy5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7k86lplx
found:  bc1pw508d6q2jwt8gny5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kzkhplx
found:  bc1pw508d6qtjjtug6y5r3zarvary0c5xw7kw508d6qejxtdg4y5r3zarvary0c5xw7kpjgplx
foun

In [80]:
def cycle_through_five_char_and_match_four(address, digit):
    hrp, data = bech32.bech32_decode(an_address)
    checksum = bech32.bech32_create_checksum(hrp, data)
    for i in range(32):
        # Change a byte
        data[digit] = i
        for j in range(4):
            data[digit+2] = j
            for k in range(32):
                data[digit+4] = k
                for l in range(32):
                    data[digit+6] = l
                    for m in range(32):
                        data[digit+8] = m
                        # Compute the checksum
                        check1 = bech32.bech32_create_checksum(hrp, data)
                        # Check the last digit
                        if (check1[-4:] == checksum[-4:]):
                            print('found: ', bech32.bech32_encode(hrp, data))

In [81]:
%time cycle_through_five_char_and_match_four(an_address, 8)

CPU times: user 7min 3s, sys: 823 ms, total: 7min 4s
Wall time: 7min 5s


huh?

Extrapolating out, looks like it would take ~45min to do a full run of 5 alterations and we should expect ~32 addresses with the same last 4 characters.

it would take ~24 hours to do a full run of 6 alterations and we would expect to see ~32 addresses with the same last 5 characters, and this would be the same timescale to find collisions of the full last 6 characters. And unlike base58Check, the least significant digits in a bech32 address won't have changed when we do this brute-force algorithm, so once the checksum matches, we can loop over any arbitrary set of characters in the data section of the address.

In [79]:
hrp, data = bech32.bech32_decode(an_address)
checksum = bech32.bech32_create_checksum(hrp, data)
print(checksum)
data[1] += 1
checksum = bech32.bech32_create_checksum(hrp, data)
print(checksum)
data = [0]
checksum = bech32.bech32_create_checksum(hrp, data)
print(checksum)
data = [1]
checksum = bech32.bech32_create_checksum(hrp, data)
print(checksum)
data = [2]
checksum = bech32.bech32_create_checksum(hrp, data)


[30, 8, 3, 1, 31, 6]
[23, 4, 26, 14, 18, 18]
[5, 2, 1, 8, 3, 28]
[24, 20, 21, 29, 30, 14]
[22, 7, 0, 11, 16, 17]


9

## Summary

2 ** 4

In [72]:
2 ** 4

16